# [Simple Models]

*KU LeeDongGyu*

### Contents

1. MLP & Data Import
```
1) Size = 64
2) Size = 48.
```
2. Simple CNN
```
1) Size = 64
2) Size = 48
```
3. Size = 48 , No Early Stopping
```
1) Epoch = 50
      (1) MLP
      (2) Simple CNN
2) Epoch = 100
      (1) MLP
      (2) Simple CNN
```


### Module

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd /content/drive/My Drive/Colab Notebooks/project

/content/drive/My Drive/Colab Notebooks/project


In [0]:
import sys

In [4]:
# 모듈로 받을 경로 확인
sys.path

['',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython']

In [0]:
# 내 노트북이 아닌, 전산실 컴퓨터의 colab에서 돌렸으므로, 다시돌리려면 경로 수정할것!
sys.path.append("/content/drive/My Drive/Colab Notebooks/project")

In [6]:
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

In [0]:
from f1score import macro_f1score, weighted_f1score

## 1. MLP & Data Import
---

In [0]:
###################### 1. model - MLP ######################

# data import
x_train = pd.read_csv("mydata/X_train.csv",header=0,index_col=0)
x_valid = pd.read_csv("mydata/X_private_test.csv",header=0,index_col=0)
x_test = pd.read_csv("mydata/X_public_test.csv",header=0,index_col=0)
y_train = pd.read_csv("mydata/y_train.csv",header=0,index_col=0)
y_valid = pd.read_csv("mydata/y_private_test.csv",header=0,index_col=0)
y_test = pd.read_csv("mydata/y_public_test.csv",header=0,index_col=0)

In [0]:
# data handling
x_train = np.array(x_train).reshape([-1,48,48,3]) 
x_valid = np.array(x_valid).reshape([-1,48,48,3]) 
x_test = np.array(x_test).reshape([-1,48,48,3]) 

y_train=to_categorical(y_train) # one hot encoding
y_valid=to_categorical(y_valid)
y_test=to_categorical(y_test)

In [0]:
size=64
x_train = np.array(x_train).reshape([-1,48,48,3])

In [11]:
x_train.shape

(28698, 48, 48, 3)

In [0]:
x_train_zoom = np.zeros([x_train.shape[0],size,size,3],dtype="float32")

for i in range(x_train.shape[0]):
    x_train_zoom[i,:] = cv2.resize(x_train[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,3) /255

In [13]:
x_train_zoom.shape

(28698, 64, 64, 3)

In [0]:
x_train = x_train / 255 # x_train, x_train_zoom 모두 0과 1사이에 값이 위치하게 설정.

In [0]:
x_valid = np.array(x_valid).reshape([-1,48,48,3])
x_valid_zoom = np.zeros([x_valid.shape[0],size,size,3],dtype="float32")
for i in range(x_valid.shape[0]):
    x_valid_zoom[i,:] = cv2.resize(x_valid[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,3) /255

In [16]:
x_valid_zoom.shape

(3589, 64, 64, 3)

In [0]:
 x_valid = x_valid / 255

In [0]:
x_test = np.array(x_test).reshape([-1,48,48,3])
x_test_zoom = np.zeros([x_test.shape[0],size,size,3],dtype="float32")
for i in range(x_test.shape[0]):
    x_test_zoom[i,:] = cv2.resize(x_test[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,3) / 255

In [19]:
x_test_zoom.shape

(3588, 64, 64, 3)

In [0]:
x_test = x_test / 255

### 1) Size = 64

In [0]:
# MLP model
inputs = tf.keras.Input(shape=(64,64,3))
x=tf.keras.layers.Flatten()(inputs)
x=tf.keras.layers.Dense(units=128,activation = 'relu',name='d1')(x)
x=tf.keras.layers.Dropout(0.3)(x)
x=tf.keras.layers.Dense(units=512,activation = 'relu',name='d2')(x)
x=tf.keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(units=7,activation = tf.nn.softmax,name='d3')(x)

In [0]:
model = tf.keras.Model(inputs=inputs,outputs=outputs)

In [47]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 12288)             0         
_________________________________________________________________
d1 (Dense)                   (None, 128)               1572992   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
d2 (Dense)                   (None, 512)               66048     
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
d3 (Dense)                   (None, 7)                 3591

In [0]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',metrics=['accuracy',macro_f1score,weighted_f1score])

In [0]:
early_stopping = EarlyStopping(monitor='val_macro_f1score', patience=3, verbose=1,mode='max')


# early stopping의 옵션은 다음과 같다.
# monitor : 관찰하고자 하는 항목. 'val_loss', 'val_acc' 가 기본이고, 내가 정의하는 옵션을 기준으로 삼을수도 있다.
# mode라는 옵션을 통해, 최소를 정할지 최대를 정할지 구체화해서 정할 수 있으며, mode옵션을 주지 않으면 auto가 default이므로, 알아서 정해줌
# 우리는 mode='max'임.

# min_delta : 개선되고 있다고 판단하기 위한 최소 변화량을 나타낸다.
# 만약 변화량이 min_delta보다 적은 경우에는 개선이 없다고 판단한다.

# patience : 개선이 없다고 바로 종료하지 않고 개선이 없는 epoch를 얼마나 기다려 줄 것인 가를 지정한다.
# 만약 10이라고 지정하면 개선이 없는 epoch가 10회가 되도록 지속될 경우 학습일 종료한다.
# 데이터의 형태에 따라 잡는 방법이 다르지만, 보통 1~10의 숫자를 택한다.


# verbose : 얼마나 자세하게 정보를 표시할 것인가를 지정한다. (0, 1, 2)

# mode : 관찰 항목에 대해 개선이 없다고 판단하기 위한 기준을 지정한다. - 다음과 같은 하위항목(auto,min,max)을 따른다.
# 예를 들어 관찰 항목이 ‘val_loss’인 경우에는 감소되는 것이 멈출 때 종료되어야 하므로, ‘min’으로 설정됨.
    # auto : 관찰하는 이름에 따라 자동으로 지정한다.
    # min : 관찰하고 있는 항목이 감소되는 것을 멈출 때 종료한다.
    # max : 관찰하고 있는 항목이 증가되는 것을 멈출 때 종료한다.

In [50]:
model.fit(x_train_zoom,y_train,batch_size=128, validation_data=(x_valid_zoom,y_valid) , epochs=100, callbacks=[early_stopping]) 

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 4s 141us/sample - loss: 1.8652 - acc: 0.2426 - macro_f1score: 0.0054 - weighted_f1score: 0.0013 - val_loss: 1.7960 - val_acc: 0.2449 - val_macro_f1score: 0.0018 - val_weighted_f1score: 4.4255e-04
Epoch 2/100
28698/28698 [==============================] - 2s 58us/sample - loss: 1.8081 - acc: 0.2514 - macro_f1score: 0.0037 - weighted_f1score: 9.4604e-04 - val_loss: 1.8167 - val_acc: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 3/100
28698/28698 [==============================] - 2s 60us/sample - loss: 1.8158 - acc: 0.2514 - macro_f1score: 4.0312e-04 - weighted_f1score: 8.7554e-05 - val_loss: 1.8156 - val_acc: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 4/100
28698/28698 [==============================] - 2s 60us/sample - loss: 1.8110 - acc: 0.2514 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_

In [51]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test_zoom,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 0s 48us/sample - loss: 1.8107 - acc: 0.2494 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00

Accuracy: 0.2494, Macro F1 Score: 0.0000, Weighted F1 Score: 0.0000


### 2) Size = 48

In [0]:
# MLP model
inputs = tf.keras.Input(shape=(48,48,3))
x=tf.keras.layers.Flatten()(inputs)
x=tf.keras.layers.Dense(units=128,activation = 'relu',name='d1')(x)
x=tf.keras.layers.Dropout(0.3)(x)
x=tf.keras.layers.Dense(units=512,activation = 'relu',name='d2')(x)
x=tf.keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(units=7,activation = tf.nn.softmax,name='d3')(x)

In [0]:
model = tf.keras.Model(inputs=inputs,outputs=outputs)

In [59]:
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 6912)              0         
_________________________________________________________________
d1 (Dense)                   (None, 128)               884864    
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
d2 (Dense)                   (None, 512)               66048     
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
d3 (Dense)                   (None, 7)                 3591

In [0]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',metrics=['accuracy',macro_f1score,weighted_f1score])

In [0]:
early_stopping = EarlyStopping(monitor='val_macro_f1score', patience=3, verbose=1,mode='max')


In [62]:
model.fit(x_train,y_train,batch_size=128, validation_data=(x_valid,y_valid) , epochs=100, callbacks=[early_stopping]) 

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 2s 83us/sample - loss: 1.8522 - acc: 0.2421 - macro_f1score: 0.0018 - weighted_f1score: 3.5235e-04 - val_loss: 1.8154 - val_acc: 0.2455 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 2/100
28698/28698 [==============================] - 2s 76us/sample - loss: 1.8108 - acc: 0.2514 - macro_f1score: 3.1746e-05 - weighted_f1score: 9.6726e-06 - val_loss: 1.8158 - val_acc: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 3/100
28698/28698 [==============================] - 2s 75us/sample - loss: 1.8096 - acc: 0.2513 - macro_f1score: 6.5727e-04 - weighted_f1score: 1.5112e-04 - val_loss: 1.8155 - val_acc: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 4/100
28698/28698 [==============================] - 2s 75us/sample - loss: 1.8107 - acc: 0.2514 - macro_f1score: 2.0378e-04 - weighted_f1score: 5.2894

In [63]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 1s 163us/sample - loss: 1.8106 - acc: 0.2494 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00

Accuracy: 0.2494, Macro F1 Score: 0.0000, Weighted F1 Score: 0.0000


## 2. Simple CNN
---

In [0]:
# FER2013 데이터를 들어가기 앞서, 주의해야할 사항이 있다.
# 기본이기도 하지만, 데이터가 Label에 대해 균형이어야 좋은 performance를 낸다. 이는 CNN에서 특히 더그런다.
# 내가가진 데이터는 불균형이므로, 불균형문제를 해결하는게 옳다.
# 이 방법은 ADASYN, SMOTE, GAN, AutoEncoder등으로 해결할 수 있으며, 이에 대한 개념을 훗날 배우고 더 적용시켜보도록 한다.
# 또한, 불균형이기에 F1 score도 같이 제시함을 잊지 않는다.

###################### 2. model - CNN Simple ######################


In [0]:
epochs = 100
classes = y_test.shape[1]
size=64

input_shape = (size,size,3)
batch_size = 128
kernel_size = (3,3) 
filters = 64 
dropout = 0.3 

### 1) Size 64

In [22]:
# CNN model
cnn_model = tf.keras.models.Sequential()
cnn_model.add(tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size,
                                     activation='relu', input_shape=input_shape, strides = (1,1) , name='Conv2D_layer1'))
cnn_model.add(tf.keras.layers.MaxPooling2D((2, 2), name='Maxpooling1_2D'))
cnn_model.add(tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size,
                                     activation='relu', strides = (1,1) , name='Conv2D_layer2'))
cnn_model.add(tf.keras.layers.MaxPooling2D((2, 2), name='Maxpooling2_2D'))
cnn_model.add(tf.keras.layers.Flatten(name='Flatten'))
cnn_model.add(tf.keras.layers.Dropout(dropout))
cnn_model.add(tf.keras.layers.Dense(64, activation='relu', name='Hidden_layer'))
cnn_model.add(tf.keras.layers.Dense(classes, activation='softmax', name='Output_layer'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [23]:
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv2D_layer1 (Conv2D)       (None, 62, 62, 64)        1792      
_________________________________________________________________
Maxpooling1_2D (MaxPooling2D (None, 31, 31, 64)        0         
_________________________________________________________________
Conv2D_layer2 (Conv2D)       (None, 29, 29, 64)        36928     
_________________________________________________________________
Maxpooling2_2D (MaxPooling2D (None, 14, 14, 64)        0         
_________________________________________________________________
Flatten (Flatten)            (None, 12544)             0         
_________________________________________________________________
dropout (Dropout)            (None, 12544)             0         
_________________________________________________________________
Hidden_layer (Dense)         (None, 64)                8

In [0]:
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy',
                  metrics=['accuracy',macro_f1score,weighted_f1score])

In [0]:
early_stopping = EarlyStopping(monitor='val_macro_f1score', patience=3, verbose=1,mode='max')

In [26]:
cnn_model.fit(x_train_zoom,y_train, validation_data=(x_valid_zoom,y_valid) ,batch_size=batch_size,epochs=epochs,callbacks=[early_stopping])

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 10s 336us/sample - loss: 1.6711 - acc: 0.3395 - macro_f1score: 0.0525 - weighted_f1score: 0.0102 - val_loss: 1.5394 - val_acc: 0.4205 - val_macro_f1score: 0.1052 - val_weighted_f1score: 0.0198
Epoch 2/100
28698/28698 [==============================] - 3s 118us/sample - loss: 1.4472 - acc: 0.4535 - macro_f1score: 0.1951 - weighted_f1score: 0.0353 - val_loss: 1.3941 - val_acc: 0.4753 - val_macro_f1score: 0.2065 - val_weighted_f1score: 0.0387
Epoch 3/100
28698/28698 [==============================] - 3s 118us/sample - loss: 1.3439 - acc: 0.4901 - macro_f1score: 0.2638 - weighted_f1score: 0.0467 - val_loss: 1.3421 - val_acc: 0.4804 - val_macro_f1score: 0.2763 - val_weighted_f1score: 0.0484
Epoch 4/100
28698/28698 [==============================] - 3s 118us/sample - loss: 1.2708 - acc: 0.5171 - macro_f1score: 0.3097 - weighted_f1score: 0.0541 - val_loss: 1.3045 - val_acc: 0.4993 - val

In [27]:
_, acc, mac_f1, wei_f1 = cnn_model.evaluate(x_test_zoom,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 0s 60us/sample - loss: 1.8693 - acc: 0.5223 - macro_f1score: 0.4650 - weighted_f1score: 0.0731

Accuracy: 0.5223, Macro F1 Score: 0.4650, Weighted F1 Score: 0.0731


### 2) Size = 48

In [0]:
epochs = 100
classes = y_test.shape[1]
size=48

input_shape = (size,size,3)
batch_size = 128
kernel_size = (3,3) 
filters = 64 
dropout = 0.3 

In [0]:
# CNN model
cnn_model2 = tf.keras.models.Sequential()
cnn_model2.add(tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size,
                                     activation='relu', input_shape=input_shape, strides = (1,1) , name='Conv2D_layer1'))
cnn_model2.add(tf.keras.layers.MaxPooling2D((2, 2), name='Maxpooling1_2D'))
cnn_model2.add(tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size,
                                     activation='relu',  strides = (1,1) , name='Conv2D_layer2'))
cnn_model2.add(tf.keras.layers.MaxPooling2D((2, 2), name='Maxpooling2_2D'))
cnn_model2.add(tf.keras.layers.Flatten(name='Flatten'))
cnn_model2.add(tf.keras.layers.Dropout(dropout))
cnn_model2.add(tf.keras.layers.Dense(64, activation='relu', name='Hidden_layer'))
cnn_model2.add(tf.keras.layers.Dense(classes, activation='softmax', name='Output_layer'))

In [32]:
cnn_model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv2D_layer1 (Conv2D)       (None, 46, 46, 64)        1792      
_________________________________________________________________
Maxpooling1_2D (MaxPooling2D (None, 23, 23, 64)        0         
_________________________________________________________________
Conv2D_layer2 (Conv2D)       (None, 21, 21, 64)        36928     
_________________________________________________________________
Maxpooling2_2D (MaxPooling2D (None, 10, 10, 64)        0         
_________________________________________________________________
Flatten (Flatten)            (None, 6400)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 6400)              0         
_________________________________________________________________
Hidden_layer (Dense)         (None, 64)               

In [0]:
cnn_model2.compile(optimizer='adam', loss='categorical_crossentropy',
                  metrics=['accuracy',macro_f1score,weighted_f1score])

In [0]:
early_stopping = EarlyStopping(monitor='val_macro_f1score', patience=3, verbose=1,mode='max')

In [35]:
cnn_model2.fit(x_train,y_train, validation_data=(x_valid,y_valid) ,batch_size=batch_size,epochs=epochs,callbacks=[early_stopping])

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 5s 183us/sample - loss: 1.6986 - acc: 0.3239 - macro_f1score: 0.0436 - weighted_f1score: 0.0084 - val_loss: 1.5679 - val_acc: 0.3865 - val_macro_f1score: 0.1304 - val_weighted_f1score: 0.0241
Epoch 2/100
28698/28698 [==============================] - 5s 174us/sample - loss: 1.4966 - acc: 0.4278 - macro_f1score: 0.1567 - weighted_f1score: 0.0289 - val_loss: 1.4843 - val_acc: 0.4461 - val_macro_f1score: 0.1761 - val_weighted_f1score: 0.0325
Epoch 3/100
28698/28698 [==============================] - 5s 173us/sample - loss: 1.4054 - acc: 0.4665 - macro_f1score: 0.2157 - weighted_f1score: 0.0387 - val_loss: 1.3798 - val_acc: 0.4798 - val_macro_f1score: 0.2172 - val_weighted_f1score: 0.0389
Epoch 4/100
28698/28698 [==============================] - 5s 173us/sample - loss: 1.3454 - acc: 0.4921 - macro_f1score: 0.2534 - weighted_f1score: 0.0448 - val_loss: 1.3795 - val_acc: 0.4625 - val_

In [36]:
_, acc, mac_f1, wei_f1 = cnn_model2.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 0s 71us/sample - loss: 1.2895 - acc: 0.5371 - macro_f1score: 0.4326 - weighted_f1score: 0.0689

Accuracy: 0.5371, Macro F1 Score: 0.4326, Weighted F1 Score: 0.0689


## 3. Size = 48 , No Early Stopping
---

### 1) Epoch = 50

#### (1) MLP

In [0]:
# MLP model
inputs = tf.keras.Input(shape=(48,48,3))
x=tf.keras.layers.Flatten()(inputs)
x=tf.keras.layers.Dense(units=128,activation = 'relu',name='d1')(x)
x=tf.keras.layers.Dropout(0.3)(x)
x=tf.keras.layers.Dense(units=512,activation = 'relu',name='d2')(x)
x=tf.keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(units=7,activation = tf.nn.softmax,name='d3')(x)

In [0]:
model = tf.keras.Model(inputs=inputs,outputs=outputs)

In [0]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',metrics=['accuracy',macro_f1score,weighted_f1score])

In [40]:
model.fit(x_train,y_train,batch_size=128, validation_data=(x_valid,y_valid) , epochs=50) 

Train on 28698 samples, validate on 3589 samples
Epoch 1/50
28698/28698 [==============================] - 4s 134us/sample - loss: 1.8588 - acc: 0.2440 - macro_f1score: 0.0046 - weighted_f1score: 8.7411e-04 - val_loss: 1.7995 - val_acc: 0.2449 - val_macro_f1score: 0.0024 - val_weighted_f1score: 5.9226e-04
Epoch 2/50
28698/28698 [==============================] - 4s 129us/sample - loss: 1.8113 - acc: 0.2514 - macro_f1score: 2.3954e-04 - weighted_f1score: 5.1452e-05 - val_loss: 1.8154 - val_acc: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 3/50
28698/28698 [==============================] - 4s 127us/sample - loss: 1.8111 - acc: 0.2514 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8153 - val_acc: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 4/50
28698/28698 [==============================] - 4s 127us/sample - loss: 1.8110 - acc: 0.2514 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00

In [41]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 0s 53us/sample - loss: 1.8112 - acc: 0.2494 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00

Accuracy: 0.2494, Macro F1 Score: 0.0000, Weighted F1 Score: 0.0000


#### (2) Simple CNN

In [0]:
# early stopping 없이 
epochs = 50
classes = y_test.shape[1]
size=48

input_shape = (size,size,3)
batch_size = 128
kernel_size = (3,3) 
filters = 64 
dropout = 0.3 

In [0]:
# CNN model
cnn_model2 = tf.keras.models.Sequential()
cnn_model2.add(tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size,
                                     activation='relu', input_shape=input_shape, strides = (1,1) , name='Conv2D_layer1'))
cnn_model2.add(tf.keras.layers.MaxPooling2D((2, 2), name='Maxpooling1_2D'))
cnn_model2.add(tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size,
                                     activation='relu',  strides = (1,1) , name='Conv2D_layer2'))
cnn_model2.add(tf.keras.layers.MaxPooling2D((2, 2), name='Maxpooling2_2D'))
cnn_model2.add(tf.keras.layers.Flatten(name='Flatten'))
cnn_model2.add(tf.keras.layers.Dropout(dropout))
cnn_model2.add(tf.keras.layers.Dense(64, activation='relu', name='Hidden_layer'))
cnn_model2.add(tf.keras.layers.Dense(classes, activation='softmax', name='Output_layer'))

In [0]:
cnn_model2.compile(optimizer='adam', loss='categorical_crossentropy',
                  metrics=['accuracy',macro_f1score,weighted_f1score])

In [45]:
cnn_model2.fit(x_train,y_train, validation_data=(x_valid,y_valid) ,batch_size=batch_size,epochs=epochs)

Train on 28698 samples, validate on 3589 samples
Epoch 1/50
28698/28698 [==============================] - 5s 191us/sample - loss: 1.6717 - acc: 0.3387 - macro_f1score: 0.0563 - weighted_f1score: 0.0113 - val_loss: 1.5254 - val_acc: 0.4230 - val_macro_f1score: 0.1236 - val_weighted_f1score: 0.0237
Epoch 2/50
28698/28698 [==============================] - 5s 184us/sample - loss: 1.4779 - acc: 0.4344 - macro_f1score: 0.1670 - weighted_f1score: 0.0313 - val_loss: 1.4173 - val_acc: 0.4614 - val_macro_f1score: 0.2073 - val_weighted_f1score: 0.0371
Epoch 3/50
28698/28698 [==============================] - 5s 183us/sample - loss: 1.3905 - acc: 0.4692 - macro_f1score: 0.2218 - weighted_f1score: 0.0397 - val_loss: 1.3668 - val_acc: 0.4706 - val_macro_f1score: 0.2502 - val_weighted_f1score: 0.0452
Epoch 4/50
28698/28698 [==============================] - 5s 183us/sample - loss: 1.3205 - acc: 0.4972 - macro_f1score: 0.2656 - weighted_f1score: 0.0471 - val_loss: 1.3071 - val_acc: 0.4987 - val_macr

In [46]:
_, acc, mac_f1, wei_f1 = cnn_model2.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 0s 63us/sample - loss: 1.6452 - acc: 0.5446 - macro_f1score: 0.4846 - weighted_f1score: 0.0750

Accuracy: 0.5446, Macro F1 Score: 0.4846, Weighted F1 Score: 0.0750


### 2) Epoch = 100

#### (1) MLP

In [0]:
# MLP model
inputs = tf.keras.Input(shape=(48,48,3))
x=tf.keras.layers.Flatten()(inputs)
x=tf.keras.layers.Dense(units=128,activation = 'relu',name='d1')(x)
x=tf.keras.layers.Dropout(0.3)(x)
x=tf.keras.layers.Dense(units=512,activation = 'relu',name='d2')(x)
x=tf.keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(units=7,activation = tf.nn.softmax,name='d3')(x)

In [0]:
model = tf.keras.Model(inputs=inputs,outputs=outputs)

In [0]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',metrics=['accuracy',macro_f1score,weighted_f1score])

In [50]:
model.fit(x_train,y_train,batch_size=128, validation_data=(x_valid,y_valid) , epochs=100) 

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 4s 138us/sample - loss: 1.8380 - acc: 0.2465 - macro_f1score: 0.0051 - weighted_f1score: 0.0012 - val_loss: 1.8014 - val_acc: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 2/100
28698/28698 [==============================] - 4s 129us/sample - loss: 1.8052 - acc: 0.2501 - macro_f1score: 0.0036 - weighted_f1score: 9.4075e-04 - val_loss: 1.8167 - val_acc: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 3/100
28698/28698 [==============================] - 4s 132us/sample - loss: 1.8106 - acc: 0.2514 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8159 - val_acc: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 4/100
28698/28698 [==============================] - 4s 129us/sample - loss: 1.8111 - acc: 0.2514 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00

In [51]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 0s 50us/sample - loss: 1.8113 - acc: 0.2494 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00

Accuracy: 0.2494, Macro F1 Score: 0.0000, Weighted F1 Score: 0.0000


#### (2) Simple CNN

In [0]:
# early stopping 없이 
epochs = 100
classes = y_test.shape[1]
size=48

input_shape = (size,size,3)
batch_size = 128
kernel_size = (3,3) 
filters = 64 
dropout = 0.3 

In [0]:
# CNN model
cnn_model2 = tf.keras.models.Sequential()
cnn_model2.add(tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size,
                                     activation='relu', input_shape=input_shape, strides = (1,1) , name='Conv2D_layer1'))
cnn_model2.add(tf.keras.layers.MaxPooling2D((2, 2), name='Maxpooling1_2D'))
cnn_model2.add(tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size,
                                     activation='relu',  strides = (1,1) , name='Conv2D_layer2'))
cnn_model2.add(tf.keras.layers.MaxPooling2D((2, 2), name='Maxpooling2_2D'))
cnn_model2.add(tf.keras.layers.Flatten(name='Flatten'))
cnn_model2.add(tf.keras.layers.Dropout(dropout))
cnn_model2.add(tf.keras.layers.Dense(64, activation='relu', name='Hidden_layer'))
cnn_model2.add(tf.keras.layers.Dense(classes, activation='softmax', name='Output_layer'))

In [0]:
cnn_model2.compile(optimizer='adam', loss='categorical_crossentropy',
                  metrics=['accuracy',macro_f1score,weighted_f1score])

In [55]:
cnn_model2.fit(x_train,y_train, validation_data=(x_valid,y_valid) ,batch_size=batch_size,epochs=epochs)

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 5s 184us/sample - loss: 1.7035 - acc: 0.3204 - macro_f1score: 0.0365 - weighted_f1score: 0.0071 - val_loss: 1.5744 - val_acc: 0.4026 - val_macro_f1score: 0.1055 - val_weighted_f1score: 0.0210
Epoch 2/100
28698/28698 [==============================] - 5s 175us/sample - loss: 1.5140 - acc: 0.4260 - macro_f1score: 0.1419 - weighted_f1score: 0.0267 - val_loss: 1.4311 - val_acc: 0.4503 - val_macro_f1score: 0.1719 - val_weighted_f1score: 0.0317
Epoch 3/100
28698/28698 [==============================] - 5s 177us/sample - loss: 1.4120 - acc: 0.4637 - macro_f1score: 0.2024 - weighted_f1score: 0.0370 - val_loss: 1.3588 - val_acc: 0.4837 - val_macro_f1score: 0.2309 - val_weighted_f1score: 0.0423
Epoch 4/100
28698/28698 [==============================] - 5s 175us/sample - loss: 1.3553 - acc: 0.4861 - macro_f1score: 0.2422 - weighted_f1score: 0.0433 - val_loss: 1.3300 - val_acc: 0.4887 - val_

In [56]:
_, acc, mac_f1, wei_f1 = cnn_model2.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 0s 64us/sample - loss: 2.0290 - acc: 0.5527 - macro_f1score: 0.4996 - weighted_f1score: 0.0781

Accuracy: 0.5527, Macro F1 Score: 0.4996, Weighted F1 Score: 0.0781
